In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
from keplergl import KeplerGl
from keplergl import documentation
import pickle
%config Completer.use_jedi = False

In [2]:
def plot_commuting(cod_municipio):
    df = pd.read_csv('data/pop/arranjos populacionais/tab01.csv', sep=';', decimal=',', thousands='.')
    cod_arranjo = str(df[df['Código do município'] == cod_municipio]['CodArranjo'].values[0])
    arranjo = str(df[df['Código do município'] == cod_municipio]['Arranjo'].values[0])
    df_commuting = pd.read_csv('data/pop/commuting/commuting.csv')
    df_commuting = df_commuting[df_commuting['Arranjo'] == arranjo]
    
    gdf = gpd.read_file(f'data/territorio/municipalities/arranjos_pop/{cod_arranjo}/arranjo_{cod_arranjo}_municipalities.zip/')

    lon = gdf.dissolve(by='CodArranjo').centroid.x[0]
    lat = gdf.dissolve(by='CodArranjo').centroid.y[0]

    minx, miny, maxx, maxy = gdf.total_bounds
    max_bound = max(abs(maxx-minx), abs(maxy-miny)) * 111
    zoom = 12.8 - np.log(max_bound)
    
    with open("data/pop/commuting/config.pkl", "rb") as a_file:
        dict_config = pickle.load(a_file)
    dict_config['config']['mapState'] = {'bearing': 0, 'dragRotate': True, 'pitch': 45, 'latitude':lat, 'longitude':lon, 'zoom':zoom}

    
    kepler_map =  kepler_map = KeplerGl(height=400)
    kepler_map.add_data(data=df_commuting, name='Commuting')
    kepler_map.add_data(data=gdf, name='Municipios_Arranjo')
    
    kepler_map.config = dict_config


    return kepler_map

In [3]:
fig = plot_commuting(cod_municipio=2927408)

C:\Users\augus\Anaconda3\envs\aulas\lib\site-packages\ipykernel_launcher.py:10: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\augus\Anaconda3\envs\aulas\lib\site-packages\ipykernel_launcher.py:11: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [4]:
fig

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '4i2fd3g', 'type': …

In [21]:
config = fig.config
with open("data/pop/commuting/config.pkl", "wb") as file:
    pickle.dump(config, file)